In [100]:
from numpy import sqrt, linspace
from matplotlib import pyplot as plt
from qutip import *
from itertools import chain

In [23]:
x = linspace(0.01,1,1000)
y = 0.1*sqrt((1-x)/x)





In [29]:
plt.plot(x,y)
plt.ylabel(r"$\sqrt{\frac{1-\gamma}{\gamma}}$")
plt.xlabel(r"$\gamma$")
plt.title(r"Plot of bound value against $\gamma$ for $\epsilon=0.1$ ")
plt.savefig("ETH BOUND Plot")

In [158]:
n= 6
initial_state = tensor([basis(2,0)]*n)
print(initial_state)
#number of spins

beta0 = lambda n,m: 0

def beta1(n,m):
    if m ==2:
        return 1
    else:
        return 0
    
    
alpha1 = Heisenberg1dRingGen(-1,1,1,n)
H1 = hamiltonian(alpha1,beta0, n)
H1=H1/H1.norm()
assert(H1.isherm)
    
energys,statex = H1.eigenstates()

Quantum object: dims = [[2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1]], shape = (64, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [159]:
def ket2dmR(state):
    n = len(state.dims[0])
    return Qobj(np.array(state)@np.array(state).T.conjugate(), dims = [[2]*n,[2]*n])

In [160]:

def get_equilibrated_dens_opV2(energys, statex, init_state):
    coefs = [init_state.overlap(state) for state in statex]
    print(coefs)
    return sum(abs(coefs[i])**2*ket2dmR(state) for i,state in enumerate(statex))


In [161]:
def get_equilibrated_dens_op(hamiltonian:Qobj, init_state:Qobj):
    energys,statex = hamiltonian.eigenstates()
    return sum(abs(state.overlap(init_state)**2)*state*state.dag() for state in statex)


In [162]:
print(get_equilibrated_dens_op(H1,initial_state))

Quantum object: dims = [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape = (64, 64), type = oper, isherm = True
Qobj data =
[[0.27538462 0.         0.         ... 0.         0.         0.04461538]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.04461538 0.         0.         ... 0.         0.         0.27538462]]


In [163]:
print(get_equilibrated_dens_opV2(energys,statex,initial_state))

assert(get_equilibrated_dens_opV2(energys,statex,initial_state)==get_equilibrated_dens_op(H1,initial_state))

[(-0.47854601324452845+0j), (0.6266618692025896+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.49999999999999883+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.23936353458184964+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.1448920743442156+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.22360679774997883+0j), 0j, 0j, 0j, 0j, 0j]
Quantum object: dims = [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape = (64, 64), type = oper, isherm = True
Qobj data =
[[0.27538462 0.         0.         ... 0.         0.         0.04461538]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.04461538 0.         0.         ... 0.         0.         0.27538462]]
[(-0.47854601324452845+0j), (0.6266618692025

In [164]:
def eq_terms(energys, statex, initial, start, end):
    
    coefs = [initial.overlap(state) for state in statex]
    return sum(abs(coefs[i]**2)*ket2dmR(statex[i]) for i in range(start,end))



def get_equilibrated_dens_op_P(energys, statex,initial,proc=4):

    number = len(statex)//proc
        
    results = [eq_terms(energys,statex,initial,i*number,(i+1)*number) for i in range(proc-1)]
    
    results.append(eq_terms(energys,statex,initial,(proc-1)*number,len(statex)))
   

    return sum(results)

In [165]:
print(get_equilibrated_dens_op_P(energys,statex,initial_state))
assert(get_equilibrated_dens_op_P(energys,statex,initial_state)==get_equilibrated_dens_opV2(energys,statex,initial_state))

Quantum object: dims = [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape = (64, 64), type = oper, isherm = True
Qobj data =
[[0.27538462 0.         0.         ... 0.         0.         0.04461538]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.04461538 0.         0.         ... 0.         0.         0.27538462]]
[(-0.47854601324452845+0j), (0.6266618692025896+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.49999999999999883+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.23936353458184964+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.1448920743442156+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.22360679774997883+0j), 0j, 0j, 0j, 0j, 0j]
